In [1]:
import os 

In [2]:
%pwd

'/home/priyanshu1303d/Projects/Text_Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/priyanshu1303d/Projects/Text_Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConifg:
    root_dir: Path
    data_path : Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [6]:
from textSummarizer.utils.common import read_yaml, create_directories
from textSummarizer.constants import *

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        print(self.config.artifacts_root)
        print(type(self.config.artifacts_root))

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConifg:
        config = self.config.model_trainer

        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConifg(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
        )

        return model_trainer_config



In [8]:
from transformers import TrainingArguments , Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer , AutoModelForSeq2SeqLM
import torch
from datasets import load_from_disk, load_dataset

/home/priyanshu1303d/anaconda3/envs/textS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-22 12:14:44.630969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740206684.650302   34561 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740206684.655830   34561 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 12:14:44.674687: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in perform

[2025-02-22 12:14:48,018 : INFO : config  : PyTorch version 2.6.0 available.]
[2025-02-22 12:14:48,021 : INFO : config  : TensorFlow version 2.18.0 available.]


In [ ]:
class ModelTrainer:
    def __init__(self , config = ModelTrainerConifg):
        self.config = config 

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading dataset 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs=self.config.num_train_epochs, 
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, 
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, 
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, 
            eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
           model=model_pegasus,
           args = trainer_args,
           tokenizer= tokenizer,
           data_collator= seq2seq_data_collator,
           train_dataset= dataset_samsum_pt['test'],  ## doing for test later on we will do it for train dataset
           eval_dataset= dataset_samsum_pt['validation'],
        )

        trainer.train()

        #save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus_samsum_model"))
        #save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))



In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-02-22 12:14:48,296 : INFO : common  : yaml file config/config.yaml was read succesfully]
[2025-02-22 12:14:48,298 : INFO : common  : yaml file params.yaml was read succesfully]
artifacts
<class 'str'>
[2025-02-22 12:14:48,300 : INFO : common  : Created directory at : artifacts]
[2025-02-22 12:14:48,300 : INFO : common  : Created directory at : artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/priyanshu1303d/anaconda3/envs/textS/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/priyanshu1303d/anaconda3/envs/textS/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/tmp/ipykernel_34561/1062751576.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
